In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import requests
import pprint

site_file = open('job_ad_sites.txt')
sites = site_file.readlines()

for site in sites:
    page = requests.get(site)
    soup = BeautifulSoup(page.content, 'html.parser')

https://www.monster.com/jobs/search/?q=Software-Developer&where=hong-kong

http://pythonjobs.github.io/

https://remote.co/remote-jobs/developer/

https://hk.indeed.com/jobs?q=software+developer&l=hong+kong

